# Read the Data File

In [250]:
import pandas as pd 

# define the path of the csv file 
file_path = r'C:\Users\Maryam\Desktop\Job applications\Portfolio\DataSets\Uncleaned_DS_Jobs.csv'

# convert the table to a dataframe
df = pd.read_csv(file_path)  

# Job Classification 

In [ ]:
# in this part of the code, a function is developed to classify the positions into the main categories we are intrested in and assign everything else to a categoty called others 
# in order to filter the data for these position at the end of the file 
# calssify the positions into the main categories which are: software enginer, data scientist, data analyst, data engineer and machine learning engineer 

def job_classification(row):
    if 'Software Engineer' in row:
        return 'Software Engineer'  
    elif 'Data' in row and 'Scientist' in row:
        return 'Data Scientist'
    elif 'Data' in row and 'Analyst' in row:
        return 'Data Analyst'
    elif 'Data' in row and 'Engineer' in row:
        return 'Data Engineer'
    elif 'Machine' in row and 'Learning' in row and 'Engineer' in row:
        return 'Machine Learning Engineer'
    else:
        return 'Others'

# apply the function to the Job Title column in the dataframe     
df['Job Classification'] = df['Job Title'].apply(job_classification)

# Level of Experience 

In [ ]:
# this function classifies the positions according to the level of experience: Expert, Senior, Mid-level and Junior 

def exp_level(row):
    if  'Lead' in row or 'Experienced' in row or 'Principal' in row:
        return 'Expert' 
    elif 'Sr' in row or 'Senior' in row: 
        return 'Senior'
    elif 'Jr' in row or 'Associate' in row: 
        return 'Junior'
    else: 
        return 'Mid-level'
    
df['Experience Level'] = df['Job Title'].apply(exp_level)

# Age of the Company 

In [251]:
# Calculate the company age from the year in which it was founded 

# since some values are missing which is indicated by (-1). The value is replaced by NA so it does not affect the calculation 
df['Founded'] = df['Founded'].replace(-1, pd.NA)

if (df['Founded'] == -1).any():
    df['Company Age'] = 'NA'
else:
    # Calculate 'Company Age' based on 'Founded' column
    df['Company Age'] = 2024 - df['Founded']

# Company Size Classification 

In [ ]:
# the size of the company is based on the number of employees. For some entries it is given as a range (x to y) or more than a specific value (10000). There are also some missing values indicated as (-1)
# or unknown

# in this part, the missing values, texts and characters are removed.  
df['Size'] = df['Size'].str.replace('-1', 'Unknown')
df['Size'] = df['Size'].str.replace('employees', '')
df['Size'] = df['Size'].str.replace('+', '')

# this function split the range into two different columns 
def split_size(row):
    if row.strip() != 'Unknown':
        if row.strip() == '10000':
            return pd.Series(['10000', '10000'])
        else:
            return pd.Series(row.split('to'))
    else: 
        return pd.Series(['Unknown', 'Unknown'])
    
df[['Min Size', 'Max Size']] = df['Size'].apply(split_size)

# once the columns are only numbers, the average size of the company is calcluated according to the function below 
def avg_size(row):
    row_1 = row['Min Size']
    row_2 = row['Max Size']
    if row_1 != 'Unknown' and row_2 != 'Unknown':
       row_1 = pd.to_numeric(row_1, errors='coerce')
       row_2 = pd.to_numeric(row_2, errors='coerce')
       average = ( row_1 +  row_2)/2
       average = int(average)
       return average
    else :
      return 'Unknown'

df['Avg Size'] = df.apply(avg_size, axis=1)

# the companies are classified into three categories according to the number of employees 
def size_classification(row):
    if  row == 'Unknown':
        return 'Unknown'
    elif int(row) >= 10000:
        return 'Large'
    elif int (row) < 100:
        return 'Small'
    else :
        return 'Medium'
    
df['Size Classification'] = df['Avg Size'].apply(size_classification)

# Average Salary Estimation 

In [ ]:
# the salaries are given as a rang for each entry so we need to split them first to find the average
# all texts and characters are removed first and then the range of the salary is divided into two columns
        
df['Salary Estimate'] = df['Salary Estimate'].str.replace('$','')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('K','')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('(Glassdoor est.)','')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('(Employer est.)','')
df[['Min Salary', 'Max Salary']] = df['Salary Estimate'].str.split('-', expand=True)

# convert the new columns into numberic values in order to calculate the average 
df['Min Salary'] = pd.to_numeric(df['Min Salary'])
df['Max Salary'] = pd.to_numeric(df['Max Salary'])

# calculate the average and convert it to integer
df['Avg Salary'] = (df['Max Salary'] + df['Min Salary'])/2
df['Avg Salary'] = df['Avg Salary'].astype(int)

# Required Technical Skills 

In [ ]:
# the set of the technical requirments are given within the job description column. In this part, we are creating a new column for each of the skills we are interested in and then assign a value of 1 or 0
# if they are mentioned within this column or not 

# Determine the skill set required for all positions 
skills = ['Python', 'aws', 'Matplotlib', 'Scikit-Learn', 'TensorFlow', 'PyTorch', 'SQL',  'Hadoop', 'Spark', 'Docker', 'Azure', 'Git', 'NLTK', ' spaCy', 'Java', 'Apache Airflow', 'Tableau', 'Power BI',
          'Web Scraping', 'APIs', 'Apache Kafka']

for skill in skills:
    df[skill] = 0  # Set the initial value to 0
    if (df['Job Description'].str.contains(skill, case=False)).any():
        df.loc[df['Job Description'].str.contains(skill, case=False), skill] = 1

# Data Filtration 

In [ ]:
# once the data has been cleaned, we will extract the entried related to the positions below and save them to another file

# here are the positions that will be included for the next step 
selected_classification = ['Data Scientist', 'Data Analyst', 'Machine Learning Engineer', 'Data Engineer', 'Software Engineer']

# select the entries to be extracted from the old dataframe
filtered_df = df[df['Job Classification'].isin(selected_classification)]

# define the location in which the new data will be saved   
new_file_path = r'C:\Users\Maryam\Desktop\Job applications\Portfolio\DataSets\Cleaned_Tech_Jobs_Data.csv'

# write the filtered dataframe to a csv file
filtered_df.to_csv(new_file_path, index=False)